In [1]:
# !pip install shap scikit-learn matplotlib pandas numpy
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from time import time

c:\dev\python\wks\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
categories = [
    "alt.atheism",
    "talk.religion.misc",
    "comp.graphics",
    "sci.space",
]

In [3]:
def size_mb(docs):
    return sum(len(s.encode("utf-8")) for s in docs) / 1e6


def load_dataset(verbose=False, remove=()):
    """Load and vectorize the 20 newsgroups dataset."""

    data_train = fetch_20newsgroups(
        subset="train",
        categories=categories,
        shuffle=True,
        random_state=42,
        remove=remove,
    )

    data_test = fetch_20newsgroups(
        subset="test",
        categories=categories,
        shuffle=True,
        random_state=42,
        remove=remove,
    )

    # order of labels in `target_names` can be different from `categories`
    target_names = data_train.target_names

    # split target in a training set and a test set
    y_train, y_test = data_train.target, data_test.target

    # Extracting features from the training data using a sparse vectorizer
    t0 = time()
    vectorizer = TfidfVectorizer(
        sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english"
    )
    X_train = vectorizer.fit_transform(data_train.data)
    duration_train = time() - t0

    # Extracting features from the test data using the same vectorizer
    t0 = time()
    X_test = vectorizer.transform(data_test.data)
    duration_test = time() - t0

    feature_names = vectorizer.get_feature_names_out()

    if verbose:
        # compute size of loaded data
        data_train_size_mb = size_mb(data_train.data)
        data_test_size_mb = size_mb(data_test.data)

        print(
            f"{len(data_train.data)} documents - "
            f"{data_train_size_mb:.2f}MB (training set)"
        )
        print(f"{len(data_test.data)} documents - {data_test_size_mb:.2f}MB (test set)")
        print(f"{len(target_names)} categories")
        print(
            f"vectorize training done in {duration_train:.3f}s "
            f"at {data_train_size_mb / duration_train:.3f}MB/s"
        )
        print(f"n_samples: {X_train.shape[0]}, n_features: {X_train.shape[1]}")
        print(
            f"vectorize testing done in {duration_test:.3f}s "
            f"at {data_test_size_mb / duration_test:.3f}MB/s"
        )
        print(f"n_samples: {X_test.shape[0]}, n_features: {X_test.shape[1]}")

    return X_train, X_test, y_train, y_test, feature_names, target_names

In [4]:
# Carregar e vetorizar os dados
X_train, X_test, y_train, y_test, feature_names, target_names = load_dataset(
    verbose=True, remove=("headers", "footers", "quotes") # remoção de metadados... que isso pode causar?
)

2034 documents - 2.43MB (training set)
1353 documents - 1.80MB (test set)
4 categories
vectorize training done in 0.156s at 15.597MB/s
n_samples: 2034, n_features: 5316
vectorize testing done in 0.095s at 18.992MB/s
n_samples: 1353, n_features: 5316


In [5]:
# Treinar o modelo de Regressão Logística
clf = LogisticRegression(max_iter=10000)
clf.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,10000
,multi_class,'deprecated'


In [6]:
# SHAP funciona melhor com dados densos ou DataFrames para algumas visualizações
# Se a memória permitir, converta X_train para denso ou para DataFrame.
# Para grandes datasets, pode ser necessário usar uma amostra (e.g., shap.sample(X_train, 100))
# Usaremos uma amostra para demonstração, caso X_train seja muito grande.
try:
    X_train_dense = X_train.toarray()
    # Se X_train for muito grande, pegue uma amostra
    if X_train.shape[0] > 500:
         print("Usando uma amostra de 100 pontos de X_train para o SHAP Explainer devido ao tamanho.")
         X_train_sampled = shap.sample(X_train_dense, 100, random_state=42)
         # Converta para DataFrame para ter nomes de features nas visualizações
         X_train_shap = pd.DataFrame(X_train_sampled, columns=feature_names)
    else:
         X_train_shap = pd.DataFrame(X_train_dense, columns=feature_names)

except MemoryError:
    print("Erro de memória ao converter X_train para denso. SHAP pode precisar de mais memória.")
    # Tentar com a matriz esparsa diretamente (pode limitar algumas visualizações)
    X_train_shap = X_train
    # NOTA: Usar matriz esparsa diretamente pode não funcionar com todas as funções de plot do SHAP
    # ou pode exigir explainer específico. LinearExplainer geralmente lida bem.

Usando uma amostra de 100 pontos de X_train para o SHAP Explainer devido ao tamanho.


In [7]:
# Criar o explainer SHAP para modelos lineares
# Passamos o modelo (clf) e os dados usados para explicação (X_train_shap)
# mask_zero=False pode ser útil se os zeros na matriz TF-IDF realmente significam ausência
explainer = shap.LinearExplainer(clf, X_train_shap, feature_perturbation="interventional")

# Calcular os valores SHAP para os dados de treino (ou uma amostra deles)
# Para Regressão Logística multi-classe (One-vs-Rest), shap_values será uma lista
# onde cada elemento corresponde aos valores SHAP para uma classe.
shap_values = explainer.shap_values(X_train_shap)

c:\dev\python\wks\.venv\Lib\site-packages\shap\explainers\_linear.py:99: FutureWarning: The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, maskers.Partition or maskers.Impute).
  warnings.warn(wmsg, FutureWarning)


In [8]:
# A média dos valores absolutos de SHAP indica a importância global da feature
# mean_abs_shap = np.abs(shap_values).mean(axis=(0,1)) # Média sobre amostras E classes

mean_abs_shap = np.abs(shap_values).mean(axis=(1,0)) # Média sobre amostras E classes

# mean_abs_shap = np.abs(shap_values) # Média sobre amostras E classes

# print(shap_values.shape[1])

# mean_abs_shap = mean_abs_shap.mean(axis=(0,1))

print(mean_abs_shap)

# print(shap_values.shape[0])
# Total Rows: 
# print(mean_abs_shap.shape[0])
#Total Columns: 
# print(mean_abs_shap.shape[1])
# print(mean_abs_shap)

# print(len(feature_names))

# print(shap_values)
# print(mean_abs_shap)
print(feature_names)


shap_summary = pd.DataFrame({
    'feature': feature_names,
    'mean_abs_shap': mean_abs_shap[1:,].mean(axis=0)
})
shap_summary = shap_summary.sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

[0.00044807 0.00052357 0.00046229 0.00039393]
['00' '000' '01' ... 'zeus' 'zip' 'zoo']


In [9]:
print("\nTop 10 features por importância global SHAP (Média Absoluta):")
print(shap_summary.head(10))
print("-" * 50)


Top 10 features por importância global SHAP (Média Absoluta):
     feature  mean_abs_shap
0        zoo        0.00046
1         00        0.00046
2        000        0.00046
3         01        0.00046
4      xtian        0.00046
5      x11r4        0.00046
6        x11        0.00046
7       wwii        0.00046
8      wustl        0.00046
9  wuarchive        0.00046
--------------------------------------------------


In [10]:
print("\nGerando gráficos SHAP...")

# Gráfico de Sumário SHAP (Importância Global - Bar Plot)
# Mostra a média do valor absoluto SHAP por feature em todas as classes
plt.figure(figsize=(10, 8)) # Criar figura separada
shap.summary_plot(shap_values, X_train_shap, plot_type="bar", feature_names=feature_names, show=False, class_names=target_names)
plt.title("Importância Global das Features (Média |SHAP|)")
plt.tight_layout()
plt.savefig("shap_global_importance.png")
print("Gráfico de importância global salvo como shap_global_importance.png")
plt.close() # Fechar a figura para evitar exibição duplicada se executado interativamente


Gerando gráficos SHAP...
Gráfico de importância global salvo como shap_global_importance.png


In [37]:
# Gráfico de Sumário SHAP (Impacto por Classe - Dot Plot)
# Mostra o impacto de cada feature em cada ponto de dado para uma classe específica
# Use class_index=0 para alt.atheism, 1 para comp.graphics, etc.
for i, class_name in enumerate(target_names):
    plt.figure(figsize=(10, 8)) # Criar figura separada
    shap.summary_plot(shap_values[i], X_train_shap, feature_names=feature_names, show=False)
    # O título é adicionado automaticamente por shap.summary_plot neste caso
    plt.title(f"Impacto das Features na Classe: {class_name}")
    plt.tight_layout()
    plt.savefig(f"shap_summary_{class_name}.png")
    print(f"Gráfico de sumário SHAP para '{class_name}' salvo como shap_summary_{class_name}.png")
    plt.close() # Fechar a figura

AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.

<Figure size 1000x800 with 0 Axes>

In [ ]:
print("-" * 50)
print("\nComparação com métodos anteriores:")
print("""
1.  **Coeficientes Ponderados (plot_feature_effects):**
    * Mostra o impacto médio das features *especificamente como aprendido pelo modelo linear*. O sinal (+/-) indica a direção do impacto para cada classe.
    * É específico para modelos lineares e pondera o coeficiente pela frequência média da feature.
    * As top features podem variar ligeiramente em relação ao SHAP, pois a ponderação é diferente (SHAP considera a contribuição marginal de cada feature).

2.  **Ganho de Informação Mútua (mutual_info_classif):**
    * Mede a dependência estatística entre uma feature e a classe alvo, *independentemente do modelo*.
    * Não informa a direção (+/-) do impacto.
    * Features com alto MI podem não ser necessariamente as mais importantes *para o modelo específico* treinado (o modelo pode não ter "aprendido" a usar essa informação eficientemente, ou a informação pode ser redundante com outras features).
    * No exemplo do notebook, o MI foi calculado apenas para a classe 'alt.atheism' (binarizada), enquanto SHAP e coeficientes analisam o modelo multi-classe.

3.  **SHAP (Global - Média |SHAP|):**
    * Fornece uma medida de importância global baseada na magnitude média da contribuição de cada feature nas previsões. É mais robusto teoricamente que apenas olhar coeficientes.
    * Geralmente concorda com as features mais importantes dos coeficientes ponderados para modelos lineares, mas a ordenação e magnitude podem diferir.
    * É aplicável a qualquer modelo, não apenas lineares.

4.  **SHAP (Sumário por Classe - Dot Plot):**
    * Mostra não apenas quais features são importantes para uma classe, mas também *como* elas impactam as previsões (valores SHAP positivos geralmente aumentam a probabilidade da classe, negativos diminuem) e a *distribuição* desses impactos nos dados. Pontos coloridos indicam o valor original da feature (alto/baixo TF-IDF), mostrando se a presença/alta frequência de uma palavra empurra a previsão para ou contra aquela classe.
    * Oferece uma visão muito mais rica que os métodos anteriores sobre o comportamento do modelo.

**Conclusão:** SHAP fornece uma visão mais detalhada e teoricamente sólida da importância das features, tanto globalmente quanto para classes específicas, e como os valores das features influenciam as previsões individuais. Ele tende a corroborar as descobertas de métodos mais simples (como coeficientes) para modelos lineares, mas oferece insights adicionais e é generalizável para modelos complexos.
""")